In [18]:
import csv
from collections import namedtuple

class Context:
    def __init__(self, gen):
        self._gen = gen

    def __enter__(self):
        return next(self._gen)

    def __exit__(self, exc_type, exc_value, exc_trace_back):
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False


def context_manager(gen_func):
    def inner(*args, **kwargs):
        return Context(gen_func(*args, **kwargs))
    return inner


@context_manager
def parsed_data_iter(fname):
    print('---')
    print(f'{fname} opening...')
    print(f'---')
    file = open(fname)
    try:
        dialect = csv.Sniffer().sniff(file.read(1000))
        file.seek(0)
        reader = csv.reader(file, dialect)
        header_row = map(lambda s: s.lower(), next(reader))
        nt = namedtuple('Data', header_row)
        yield (nt(*data) for data in reader)
    finally:
        print('---')
        print(f'{fname} closing...')
        print(f'---')
        file.close()


class ExitStack:
    def __init__(self):
        self._readers = []
        self._exits = []

    def __iter__(self):
        return self

    def __next__(self):
        return [next(reader) for reader in self._readers]

    def __enter__(self):
        return self

    def enter_context(self, fname):
        ctx = parsed_data_iter(fname)
        self._exits.append(ctx.__exit__)
        self._readers.append(ctx.__enter__())

    def __exit__(self, exc_type, exc_value, exc_trace_back):
        for exit in self._exits[::-1]:
            exit(exc_type, exc_value, exc_trace_back)
        return False
